이미 만들어 둔, 문서 - 단어 행렬을 이용하여 NMF 를 이용한 토픽모델링을 합니다.

In [2]:
from lovit_textmining_dataset.navernews_10days import get_bow

x, idx_to_vocab, vocab_to_idx = get_bow(date='2016-10-20', tokenize='noun')

300,91 개의 문서가 9,774 개의 단어로 표현되어 있습니다.

In [3]:
x.shape

(30091, 9774)

사용법은 sklearn.decomposition.TruncatedSVD 와 같습니다. l1_ratio 는 L1 regularization 과 L2 regularization 의 비용의 비율입니다. alpha 는 L1 와 L2 regularization 의 비용값입니다.

하지만 학습 시간이 조금 긴 편입니다. SVD 가 16.2 초 만에 학습되었지만, NMF 는 5 분의 학습 시간이 필요합니다. 학습이 오래 걸리니 한 번 학습한 뒤, 파일을 저장해둡니다.

In [4]:
%%time

import pickle
from sklearn.decomposition import NMF

TRAIN = False
if TRAIN:
    nmf = NMF(n_components=100, l1_ratio=0.5, alpha=1.0)
    y = nmf.fit_transform(x)

    with open('./2016-10-20-nmf.pkl', 'wb') as f:
        pickle.dump(nmf, f)
    with open('./2016-10-20-nmf_y.pkl', 'wb') as f:
        pickle.dump(y, f)

else:
    # load trained model
    with open('./2016-10-20-nmf.pkl', 'rb') as f:
        nmf = pickle.load(f)
    with open('./2016-10-20-nmf_y.pkl', 'rb') as f:
        y = pickle.load(f)

CPU times: user 48 ms, sys: 28 ms, total: 76 ms
Wall time: 114 ms


NMF 가 차원을 축소한 대상은 Bag of Words 로 표현된 문서벡터 입니다. 9,774 차원으로 표현되는 문서가 100 차원으로 표현됩니다.

In [5]:
y.shape

(30091, 100)

9,774 개의 단어 역시 100 차원으로 표현됩니다. SVD 처럼 components_ 에 정보가 저장되어 있습니다.

이를 반대로 해석하면 100 개의 components 들이 각각 9,774 개의 term weight vector 로 표현된 것과 같습니다. 30091 개의 문서를 표현할 수 있는 100 개의 components 입니다. 이는 마치 topic vector 의 역할을 합니다.

In [6]:
nmf.components_.shape

(100, 9774)

components vector 를 topic_vector 로 복사해둡니다.

In [7]:
topic_vector = nmf.components_.copy()

한 component 에 대하여 weight 의 크기가 큰 순서대로 component keyword 를 선택할 수 있습니다. 일종의 labeling 입니다.

71 번 component 는 아이돌 방송 관련 토픽들 입니다.

In [8]:
def most_important_terms(topic_idx, topn=10):
    
    # sort by weight in decreasing order
    term_idxs = topic_vector[topic_idx,:].argsort()[::-1]

    # select top n terms
    if topn > 0:
        term_idxs = term_idxs[:topn]

    # form of [(idx, weight), ... ]
    weights = topic_vector[topic_idx, term_idxs]
    term_and_weights = [(t,w) for t,w in zip(term_idxs, weights)]

    # decode
    term_and_weights = [(idx_to_vocab[t], w) for t,w in term_and_weights]

    return term_and_weights

most_important_terms(71)

[('불독', 8.363253756319944),
 ('데뷔', 3.9461130931812556),
 ('걸그룹', 3.384022901865648),
 ('쇼케이스', 2.3951614337193554),
 ('키미', 2.254010318027516),
 ('형은', 2.1442961282761828),
 ('무대', 1.8579620028547768),
 ('소라', 1.7919308780063787),
 ('롤링', 1.6990087077278002),
 ('세이', 1.6359480115919014)]

1 번 components 는 미국 대선 관련 성분입니다.

In [9]:
most_important_terms(1)

[('트럼프', 14.349792812895624),
 ('클린턴', 9.743343715312415),
 ('토론', 6.1658334256197325),
 ('대선', 4.224739238313858),
 ('후보', 3.1958181275118376),
 ('힐러리', 2.7636573463602816),
 ('주장', 2.485726820649654),
 ('공화당', 2.0921033834376894),
 ('3차', 2.0437456715235274),
 ('미국', 1.8113614236301592)]

한 단어에 대하여 weight 가 큰 components 를 찾을 수도 있습니다.

In [10]:
def most_relavant_topics(term, n_topics=5, n_terms=20):
    term_idx = vocab_to_idx.get(term, -1)
    if term_idx < 0:
        return []

    # slice topic weight vector
    relavant_topics = topic_vector[:,term_idx].argsort()[::-1]

    # select top topics
    if n_topics > 0:
        relavant_topics = relavant_topics[:n_topics]

    # select important terms for each topic (component)
    topic_terms = []
    for topic_idx in relavant_topics:
        # idx, score, terms
        topic_terms.append(
            (topic_idx,
             topic_vector[topic_idx, term_idx],
             most_important_terms(topic_idx, n_terms))
        )
    
    return topic_terms

단어의 빈도수를 확인하기 위하여 term frequency matrix 에 row sum 을 하여 frequency vector 를 만듭니다.

In [11]:
import numpy as np

term_frequency = np.asarray(x.sum(axis=0)).reshape(-1)
term_frequency.shape

(9774,)

`아이오아이` 라는 단어의 weight 가 큰 components 들의 top weighted words 입니다. `아이오아이` 라는 단어와 관련된 토픽 (components) 은 크게 3 개 정도로 학습되었지만, 해당 component 들에서의 weight 는 크지 않습니다.

In [12]:
query = '아이오아이'
query_idx = vocab_to_idx[query]
print('query term = {}, frequency = {}\n'.format(query, term_frequency[query_idx]))

relevant_topics = most_relavant_topics(query)

for topic_id, relavant_score, important_terms in relevant_topics:
    terms = ' '.join([term for term, _ in important_terms])
    print('component#{} ({:.4}): {}'.format(topic_id, relavant_score, terms))

query term = 아이오아이, frequency = 270

component#71 (0.6861): 불독 데뷔 걸그룹 쇼케이스 키미 형은 무대 소라 롤링 세이 마포구 매력 101 오전 멤버들 싱글 프로듀스 강렬 20일 표현
component#11 (0.2192): 방송 출연 프로그램 이날 무대 예능 전현무 웃음 시청률 노래 지상파 김지민 오후 아프리카 광고 샤이니 한편 20일 시청자들 음악
component#60 (0.1786): 1위 트와이스 기록 스트리밍 방탄소년단 차트 누적 발표 뮤직비디오 올해 2016년 가온차트 데뷔 유튜브 최고 미니앨범 조회수 차지 1주년 20일
component#81 (0.05368): 공개 모습 캔디 화보 마음 캐릭터 매력 한편 장근석 영상 촬영 메이크업 조안 소리 기대감 예정 측은 특히 티저 기대
component#79 (0.03476): 신화 앨범 발매 13집 팬들 정규 컴백 활동 11월 그룹 콘서트 데뷔 공개 이번 예정 멤버들 곡들 19년 기대 29일


`트와이스` 가 해당 시기에 더 자주 등장한 단어였기 때문에 weight 가 훨씬 크게 학습되었습니다. component#79 는 아이돌 관련 component 임을 알 수 있습니다.

In [13]:
query = '트와이스'
query_idx = vocab_to_idx[query]
print('query term = {}, frequency = {}\n'.format(query, term_frequency[query_idx]))

relevant_topics = most_relavant_topics(query)

for topic_id, relavant_score, important_terms in relevant_topics:
    terms = ' '.join([term for term, _ in important_terms])
    print('component#{} ({:.4}): {}'.format(topic_id, relavant_score, terms))

query term = 트와이스, frequency = 655

component#60 (4.822): 1위 트와이스 기록 스트리밍 방탄소년단 차트 누적 발표 뮤직비디오 올해 2016년 가온차트 데뷔 유튜브 최고 미니앨범 조회수 차지 1주년 20일
component#81 (0.1834): 공개 모습 캔디 화보 마음 캐릭터 매력 한편 장근석 영상 촬영 메이크업 조안 소리 기대감 예정 측은 특히 티저 기대
component#19 (0.02213): 네이버 의장 대표 서비스 이사회 유럽 한성숙 이해진 글로벌 부사장 3월 내년 대표이사 김상헌 라인 내정자 의장직 업계 내정 차기
component#13 (0.009335): 차태현 김유정 영화 서현진 박보검 사랑 때문 성동일 연기 김윤혜 박근형 주지홍 구르미 호흡 달빛 감독 많이 기억 개봉 출연
component#99 (0.0): 디자인 브랜드 선보 출시 패션 모델 소재 스타일 컬러 국내 매장 블랙 컬렉션 활용 이번 재킷 다운 적용 상품 가을


'트럼프'라는 단어의 weight 가 큰 components 들의 top weighted words 입니다. component#1 과 component#39 에 집중되어 있으며, term frequency 가 크기 때문에 weight 도 큽니다.

In [14]:
query = '트럼프'
query_idx = vocab_to_idx[query]
print('query term = {}, frequency = {}\n'.format(query, term_frequency[query_idx]))

relevant_topics = most_relavant_topics(query)

for topic_id, relavant_score, important_terms in relevant_topics:
    terms = ' '.join([term for term, _ in important_terms])
    print('component#{} ({:.4}): {}'.format(topic_id, relavant_score, terms))

query term = 트럼프, frequency = 3540

component#1 (14.35): 트럼프 클린턴 토론 대선 후보 힐러리 주장 공화당 3차 미국 선거 민주당 도널드 대선후보 결과 라스베이거스 이날 지지 발언 19일
component#39 (2.226): 후보 지금 힐러리 있습니다 말씀 트럼프 이런 제가 때문 미국 사실 생각 생각합니다 클린턴 경제 발언 국가 대통령 우리 도널드
component#14 (0.02678): 중국 필리핀 두테르테 양국 남중국해 베이징 주석 정상회담 분쟁 협력 영유권 시진핑 방문 경제 성장률 투자 갈등 수출 로드 달러
component#78 (0.007459): 여성 남성 문제 남성들 혐오 여성들 사회 생각 자신 말하는 표현 임신 남자 운동 낙태 이상 차별 사건 것이다 하지
component#42 (0.004304): 영화 감독 작품 개봉 배우 연기 이야기 걷기왕 관객 관객들 심은경 제작 흥행 만복 주연 출연 스크린 럭키 배우들 영화제


NMF 는 SVD 와 활용하는 방법은 비슷합니다. component 를 학습하는 방식이 다릅니다.